In [1]:
import time
import boto3
from sagemaker import session

sm_client = boto3.client("sagemaker")
account = boto3.client("sts").get_caller_identity().get("Account")
role = f"arn:aws:iam::{account}:role/service-role/AmazonSageMakerServiceCatalogProductsUseRole"

C:\Users\tochi\anaconda3\envs\sagemaker-dev\lib\site-packages\pydantic\_internal\_fields.py:172: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[02/09/25 10:18:49] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=566572;file://C:\Users\tochi\anaconda3\envs\sagemaker-dev\lib\site-packages\botocore\credentials.py\credentials.py]8;;\:]8;id=85839;file://C:\Users\tochi\anaconda3\envs\sagemaker-dev\lib\site-packages\botocore\credentials.py#1278\1278]8;;\

sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\tochi\AppData\Local\sagemaker\sagemaker\config.yaml


[02/09/25 10:18:50] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=584453;file://C:\Users\tochi\anaconda3\envs\sagemaker-dev\lib\site-packages\botocore\credentials.py\credentials.py]8;;\:]8;id=54922;file://C:\Users\tochi\anaconda3\envs\sagemaker-dev\lib\site-packages\botocore\credentials.py#1278\1278]8;;\

### Enable Data Capture for Your Endpoint

In [2]:
# Define endpoint and S3 bucket
endpoint_name = "stock-market-prediction-endpoint"
current_time = time.strftime("%m-%d-%H-%M-%S", time.localtime())
endpoint_config_name = "stock-prediction-endpoint-config-" + current_time
prefix = "data-capture"

# Enable data capture configuration
data_capture_config = {
    "EnableCapture": True,
    "InitialSamplingPercentage": 100,
    "DestinationS3Uri": "s3://aws-portfolio-projects/snp500-data/monitoring_artifacts/",
    "CaptureOptions": [{"CaptureMode": "Input"}, {"CaptureMode": "Output"}],
    "CaptureContentTypeHeader": {
        "CsvContentTypes": ["text/csv"],
        "JsonContentTypes": ["application/json"],
    },
}

# Create a new endpoint configuration with data capture
response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "AllTraffic",
            "ModelName": "pipelines-okya27xmgwox-StockMarketPredictio-7qW5sj5SWn",
            "InitialInstanceCount": 1,
            "InstanceType": "ml.m5.large",
        }
    ],
    DataCaptureConfig=data_capture_config,
)
print(response)

# Update endpoint to enable data capture
response = sm_client.update_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name,
)

print("Data capture enabled for the endpoint.")

{'EndpointConfigArn': 'arn:aws:sagemaker:us-east-2:930627915954:endpoint-config/stock-prediction-endpoint-config-02-09-10-24-12', 'ResponseMetadata': {'RequestId': '0b6ca6a9-bed0-4ed8-abc0-e607fe3625f6', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '0b6ca6a9-bed0-4ed8-abc0-e607fe3625f6', 'content-type': 'application/x-amz-json-1.1', 'content-length': '128', 'date': 'Sun, 09 Feb 2025 16:24:13 GMT'}, 'RetryAttempts': 0}}
Data capture enabled for the endpoint.


### Baseline Data and Constraints

In [5]:
from sagemaker.model_monitor import DefaultModelMonitor

monitor = DefaultModelMonitor(
    role=role,
    instance_count=1,
    instance_type="ml.m5.large",
    volume_size_in_gb=20,
    max_runtime_in_seconds=3600,
    sagemaker_session=session.Session(),
)

# Generate a unique job name using the current timestamp
current_time = time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())
baseline_job_name = f"baseline-job-stock-model-{current_time}"

monitor.suggest_baseline(
    baseline_dataset="s3://aws-portfolio-projects/snp500-data/train_data/features.csv",
    dataset_format={"Csv": {"Header": True}},
    output_s3_uri="s3://aws-portfolio-projects/snp500-data/monitoring_artifacts/",
    job_name=baseline_job_name,
)

print(f"Baseline job started: {baseline_job_name}")

### Set up monitoring schedule

In [7]:
from sagemaker.model_monitor import DefaultModelMonitor, CronExpressionGenerator

# Initialize the model monitor
monitor = DefaultModelMonitor(
    role=role,
    instance_count=1,
    instance_type="ml.m5.large",
    volume_size_in_gb=20,
    max_runtime_in_seconds=3600,
    sagemaker_session=session.Session(),
)

# Define the monitoring schedule name
monitor_schedule_name = "stock-model-monitoring-schedule"

# Create the monitoring schedule to run hourly
monitor.create_monitoring_schedule(
    endpoint_input=endpoint_name,
    output_s3_uri="s3://aws-portfolio-projects/snp500-data/monitoring_artifacts/",
    statistics="s3://aws-portfolio-projects/snp500-data/monitoring_artifacts/statistics.json",
    constraints="s3://aws-portfolio-projects/snp500-data/monitoring_artifacts/constraints.json",
    schedule_cron_expression=CronExpressionGenerator.hourly(),
    monitor_schedule_name=monitor_schedule_name,
)

print(f"Monitoring schedule created: {monitor_schedule_name}")

[02/09/25 12:17:48] INFO     Ignoring unnecessary instance type: None.                            ]8;id=761104;file://C:\Users\tochi\anaconda3\envs\sagemaker-dev\lib\site-packages\sagemaker\image_uris.py\image_uris.py]8;;\:]8;id=847714;file://C:\Users\tochi\anaconda3\envs\sagemaker-dev\lib\site-packages\sagemaker\image_uris.py#528\528]8;;\

[02/09/25 12:17:50] INFO     Creating Monitoring Schedule with name:                       ]8;id=893866;file://C:\Users\tochi\anaconda3\envs\sagemaker-dev\lib\site-packages\sagemaker\model_monitor\model_monitoring.py\model_monitoring.py]8;;\:]8;id=30616;file://C:\Users\tochi\anaconda3\envs\sagemaker-dev\lib\site-packages\sagemaker\model_monitor\model_monitoring.py#1560\1560]8;;\
                             stock-model-monitoring-schedule                                                       

Monitoring schedule created: stock-model-monitoring-schedule


### Set up cloudwatch alarm

In [9]:
import boto3

# Initialize boto3 client for CloudWatch
cloudwatch_client = boto3.client("cloudwatch")

# Define SNS topic ARN for notifications
sns_topic_arn = "arn:aws:sns:us-east-2:930627915954:StockModelAlarmTopic"

# Create the CloudWatch alarm
response = cloudwatch_client.put_metric_alarm(
    AlarmName="StockModelConstraintViolationsAlarm",
    AlarmDescription="Triggers if constraint violations are detected for the stock prediction model.",
    ActionsEnabled=True,
    MetricName="ConstraintViolations",
    Namespace="AWS/SageMaker",
    Statistic="Sum",
    Dimensions=[
        {"Name": "MonitoringScheduleName", "Value": "stock-model-monitoring-schedule"}
    ],
    Period=300,  # 5 minutes
    EvaluationPeriods=1,
    Threshold=0,
    ComparisonOperator="GreaterThanThreshold",
    AlarmActions=[sns_topic_arn],  # Sends notifications to an SNS topic
    OKActions=[sns_topic_arn],  # Optional: Send "OK" notifications
    TreatMissingData="notBreaching",  # Ignore missing data points
)

print("CloudWatch alarm created:", response)

CloudWatch alarm created: {'ResponseMetadata': {'RequestId': '452303a9-d5a6-4125-9f71-c9964f760350', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '452303a9-d5a6-4125-9f71-c9964f760350', 'content-type': 'text/xml', 'content-length': '214', 'date': 'Sun, 09 Feb 2025 22:25:40 GMT'}, 'RetryAttempts': 0}}


In [10]:
import boto3

# Initialize SageMaker client
sagemaker_client = boto3.client("sagemaker")

# List all monitoring schedules
response = sagemaker_client.list_monitoring_schedules()

# Iterate through the monitoring schedules and delete those associated with the endpoint
for schedule in response["MonitoringScheduleSummaries"]:
    schedule_name = schedule["MonitoringScheduleName"]
    schedule_details = sagemaker_client.describe_monitoring_schedule(
        MonitoringScheduleName=schedule_name
    )

    if (
        schedule_details["EndpointName"] == "stock-market-prediction-endpoint"
    ):  # Replace with your endpoint name
        sagemaker_client.delete_monitoring_schedule(
            MonitoringScheduleName=schedule_name
        )
        print(f"Deleted monitoring schedule: {schedule_name}")

print("All associated monitoring schedules deleted.")

Deleted monitoring schedule: stock-model-monitoring-schedule
All associated monitoring schedules deleted.
